# **웹 크롤링**
Web Mining
## **1 Natural Language Processing**
자연어 분석을 위한 크롤링

In [1]:
# pip install beautifulsoup4
import os
import numpy as np
from bs4 import BeautifulSoup
moviehtmldir, moviedict = './data/Movie/movie/', {}

for filename in [f for f in os.listdir(moviehtmldir) if f[0]!='.']:
    id = filename.split('.')[0]
    f  = open(moviehtmldir + '/' + filename, encoding="ISO-8859-1")
    parsed_html   = BeautifulSoup(f.read(), "lxml")
    try:    title = parsed_html.body.h1.text       
    except: title = 'none'
    moviedict[id] = title

In [2]:
import nltk
from nltk.corpus   import stopwords
from nltk.tokenize import WordPunctTokenizer

# nltk.download('stopwords')
stoplist = stopwords.words('english')
tknzr    = WordPunctTokenizer()

def ListDocs(dirname):
    docs, titles = [], []
    for filename in [f for f in os.listdir(dirname) if str(f)[0]!='.']:
        f    = open(dirname+'/'+filename,'r')
        id_  = filename.split('.')[0].split('_')[1]
        titles.append(moviedict[id_])
        docs.append(f.read())
    return docs,titles

dir_ = './data/txt_sentoken/'
pos_textreviews, pos_titles = ListDocs(dir_ + 'pos/')
neg_textreviews, neg_titles = ListDocs(dir_ + 'neg/')
tot_textreviews = pos_textreviews + neg_textreviews
tot_titles      = pos_titles + neg_titles

In [3]:
#test tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def PreprocessTfidf(texts,stoplist=[],stem=False):
    newtexts = []
    for text in texts:
        if stem:
            tmp = [w for w in tknzr.tokenize(text) if w not in stoplist]
        else:
            tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
        newtexts.append(' '.join(tmp))
    return newtexts

vectorizer        = TfidfVectorizer(min_df=1)
processed_reviews = PreprocessTfidf(tot_textreviews,stoplist,True)
mod_tfidf         = vectorizer.fit(processed_reviews)
vec_tfidf         = mod_tfidf.transform(processed_reviews)
tfidf             = dict(zip(vectorizer.get_feature_names(),vectorizer.idf_))

In [4]:
#dump tf-idf into file
import pickle
#print mod_tfidf.get_feature_names()
print(len(processed_reviews),'--',len(mod_tfidf.get_feature_names()))
v = mod_tfidf.transform(processed_reviews)

with open('vectorizer.pk', 'wb') as fin:
      pickle.dump(mod_tfidf, fin)

file = open("vectorizer.pk",'rb')
load_tfidf =  pickle.load(file)
load_tfidf.transform(PreprocessTfidf([' '.join(['drama'])],stoplist,True))

2000 -- 39516


<1x39516 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [6]:
#test LSA
import gensim
from gensim import models

class GenSimCorpus(object):
    
    def __init__(self, texts, stoplist=[],stem=False):
        self.texts      = texts
        self.stoplist   = stoplist
        self.stem       = stem
        self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))

    def __len__(self):
        return len(self.texts)
    
    def __iter__(self):
        for tokens in self.iter_docs(self.texts, self.stoplist):
            yield self.dictionary.doc2bow(tokens)
            
    def iter_docs(self,texts, stoplist):
        for text in texts:
            if self.stem:
                yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
            else:
                yield (x for x in tknzr.tokenize(text) if x not in stoplist)

corpus      = GenSimCorpus(tot_textreviews,stoplist,True)
dict_corpus = corpus.dictionary
ntopics     = 10
lsi         = models.LsiModel(corpus, num_topics=ntopics, id2word=dict_corpus)

In [7]:
U     = lsi.projection.u
Sigma = np.eye(ntopics) * lsi.projection.s

# calculate V
V          = gensim.matutils.corpus2dense(lsi[corpus], len(lsi.projection.s)).T / lsi.projection.s
dict_words = {}
for i in range(len(dict_corpus)):
    dict_words[dict_corpus[i]] = i

/home/markbaum/Python/django111/lib/python3.6/site-packages/gensim/matutils.py:491: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


In [8]:
from collections import namedtuple

def PreprocessDoc2Vec(text,stop=[],stem=False):
    words = tknzr.tokenize(text)
    if stem:
        words_clean = [stemmer.stem(w) for w in [i.lower() for i in words if i not in stop]]
    else:
        words_clean = [i.lower() for i in words if i not in stop]
    return words_clean

In [9]:
Review = namedtuple('Review','words tags')
dir_ = './data/txt_sentoken/'
do2vecstem = False
reviews_pos = []
cnt = 0
filelist = [f   for f in os.listdir(dir_ + 'pos/') 
                if str(f)[0] != '.']

for filename in filelist:
    f = open(dir_ + 'pos/'+filename,'r')
    reviews_pos.append(Review(PreprocessDoc2Vec(f.read(),stoplist,do2vecstem),['pos_'+str(cnt)]))
    cnt += 1

In [10]:
reviews_neg = []
cnt = 0
for filename in [f for f in os.listdir(dir_ + 'neg/') if str(f)[0]!='.']:
    f = open(dir_ + 'neg/' + filename,'r')
    reviews_neg.append(Review(PreprocessDoc2Vec(f.read(),stoplist,do2vecstem),['neg_'+str(cnt)]))
    cnt+=1
tot_reviews = reviews_pos + reviews_neg

In [21]:
#define doc2vec
from gensim.models import Doc2Vec
import multiprocessing
cores     = multiprocessing.cpu_count()
vec_size  = 500
model_d2v = Doc2Vec(dm=1, dm_concat=0, vec_size=vec_size, window=10, negative=0, hs=0, min_count=1, workers=cores)

#train
numepochs= 20

#build vocab
model_d2v.build_vocab(tot_reviews)

# for epoch in range(numepochs):
#     try:
#         print('epoch %d' % (epoch))
#         model_d2v.train(tot_reviews)
#         model_d2v.train(tot_reviews, epochs=model.iter, total_examples=model.corpus_count)
#         model_d2v.alpha *= 0.99
#         model_d2v.min_alpha = model_d2v.alpha
#     except (KeyboardInterrupt, SystemExit):
#         break

In [16]:
#query
query = ['science','future','action']
#similar tfidf
#sparse matrix so the metrics transform into regular vectors before computing cosine
from sklearn.metrics.pairwise import cosine_similarity
query_vec  = mod_tfidf.transform(PreprocessTfidf([' '.join(query)],stoplist,True))
sims       = cosine_similarity(query_vec,vec_tfidf)[0]
indxs_sims = sims.argsort()[::-1]
for d in list(indxs_sims)[:5]:
    print ('sim:',sims[d],' title:',tot_titles[d])

sim: 0.1779486504573889  title: No Telling (1991)
sim: 0.1778211465665258  title: Total Recall (1990)
sim: 0.17378379866060237  title: Time Machine, The (1960)
sim: 0.16303179622390185  title: Bicentennial Man (1999)
sim: 0.160582512878302  title: Andromeda Strain, The (1971)


In [17]:
#LSA query
def TransformWordsListtoQueryVec(wordslist,dict_words,stem=False):
    q = np.zeros(len(dict_words.keys()))
    for w in wordslist:
        if stem:
            q[dict_words[stemmer.stem(w)]]=1.
        else:
            q[dict_words[w]] = 1.
    return q

q    = TransformWordsListtoQueryVec(query,dict_words,True)
qk   = np.dot(np.dot(q,U),Sigma)
sims = np.zeros(len(tot_textreviews))
for d in range(len(V)):
    sims[d] = np.dot(qk,V[d])

indxs_sims = np.argsort(sims)[::-1]  
for d in list(indxs_sims)[:5]:
    print('sim:',sims[d],' doc:',tot_titles[d])

sim: 4.030303923978979  doc: Star Wars: Episode I - The Phantom Menace (1999)
sim: 3.4188592993754714  doc: Alien³ (1992)
sim: 3.4087585167111683  doc: Rocky Horror Picture Show, The (1975)
sim: 2.995740915887351  doc: Starship Troopers (1997)
sim: 2.8594701640674547  doc: Wild Things (1998)


In [24]:
# #doc2vec query
# #force inference to get the same result
# model_d2v.random = np.random.RandomState(1)
# query_docvec = model_d2v.infer_vector(PreprocessDoc2Vec(' '.join(query),stoplist,do2vecstem))

# model_d2v.docvecs.most_similar([query_docvec], topn=3)
# reviews_related = model_d2v.docvecs.most_similar([query_docvec], topn=5)
# for review in reviews_related:
#     print('relevance:',review[1],'  title:',tot_titles[review[0]]) 

## 2. Post Processing (데이터 전처리)

In [25]:
import os
import numpy as np

#get titles
from bs4 import BeautifulSoup
moviehtmldir = './data/Movie/movie/'
moviedict = {}

In [26]:
for filename in [f for f in os.listdir(moviehtmldir) if f[0]!='.']:
    id_ = filename.split('.')[0]
    f = open(moviehtmldir + '/' + filename, encoding="ISO-8859-1")
    parsed_html = BeautifulSoup(f.read())
    try:
        title = parsed_html.body.h1.text
    except:
        title = 'none'
    moviedict[id_] = title

In [27]:
def ListDocs(dirname):
    docs = []
    titles = []
    for filename in [f for f in os.listdir(dirname) if str(f)[0]!='.']:
        f = open(dirname+'/'+filename,'r', encoding="ISO-8859-1")
        id = filename.split('.')[0].split('_')[1]
        titles.append(moviedict[id])
        docs.append(f.read())
    return docs,titles

dir_ = './data/txt_sentoken/'
pos_textreviews,pos_titles = ListDocs(dir_ + 'pos/')
neg_textreviews,neg_titles = ListDocs(dir_ + 'neg/')
tot_textreviews = pos_textreviews+neg_textreviews
tot_titles = pos_titles+neg_titles

In [28]:
#LDA
import gensim.models
from gensim import models
from nltk.tokenize import RegexpTokenizer
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [39]:
class GenSimCorpus(object):
    
    def __init__(self, texts, stoplist=[],bestwords=[],stem=False):
        self.texts, self.stem, self.stoplist, self.bestwords = texts, stem, stoplist, bestwords
        self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))

    def __len__(self): return len(self.texts)

    def __iter__(self):
        for tokens in self.iter_docs(self.texts, self.stoplist):
            yield self.dictionary.doc2bow(tokens)
            
    def iter_docs(self,texts, stoplist):
        for text in texts:
            if self.stem: yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
            else:
                if len(self.bestwords)>0: yield (x for x in tknzr.tokenize(text) if x in self.bestwords)
                else: yield (x for x in tknzr.tokenize(text) if x not in stoplist)                      

num_topics = 10
corpus     = GenSimCorpus(tot_textreviews, stoplist,[],False)
dict_lda   = corpus.dictionary
lda        = models.LdaModel(corpus, num_topics=num_topics, id2word=dict_lda,passes=10, iterations=50)
lda.show_topics(num_topics=num_topics)

[(0,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"movie" + 0.000*"film" + 0.000*"one" + 0.000*"like" + 0.000*"even" + 0.000*"get"'),
 (1,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"film" + 0.000*"movie" + 0.000*"one" + 0.000*"even" + 0.000*"get" + 0.000*"time"'),
 (2,
  '0.716*" " + 0.037*"\n" + 0.022*"\'" + 0.011*"-" + 0.002*"film" + 0.002*"movie" + 0.001*"one" + 0.001*"like" + 0.001*"even" + 0.001*"good"'),
 (3,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"film" + 0.000*"one" + 0.000*"movie" + 0.000*"like" + 0.000*"get" + 0.000*"time"'),
 (4,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"movie" + 0.000*"one" + 0.000*"film" + 0.000*"like" + 0.000*"get" + 0.000*"even"'),
 (5,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"film" + 0.000*"one" + 0.000*"movie" + 0.000*"like" + 0.000*"get" + 0.000*"good"'),
 (6,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"film" + 0.000*"movie" + 0.000*"one" + 0.000*"e

In [40]:
import copy
#filter out very common words like mobie and film or very unfrequent terms
out_ids  = [tokenid for tokenid, docfreq in dict_lda.dfs.items() if docfreq > 1000 or docfreq < 3 ]
dict_lfq = copy.deepcopy(dict_lda)
dict_lfq.filter_tokens(out_ids)
dict_lfq.compactify()
corpus   = [dict_lfq.doc2bow(tknzr.tokenize(text)) for text in tot_textreviews]

lda_lfq = models.LdaModel(corpus, num_topics=num_topics, id2word=dict_lfq,passes=10, iterations=50,alpha=0.01,eta=0.01)
for t in range(num_topics):
    print('topic ',t,'  words: ',lda_lfq.print_topic(t,topn=10))

/home/markbaum/Python/django111/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


topic  0   words:  0.005*"see" + 0.004*"bad" + 0.004*"/" + 0.003*"really" + 0.003*"could" + 0.003*"star" + 0.003*"+" + 0.003*"plot" + 0.003*"films" + 0.003*"characters"
topic  1   words:  0.004*"characters" + 0.004*"best" + 0.003*"scene" + 0.003*"life" + 0.003*"could" + 0.003*"great" + 0.003*"action" + 0.003*"performance" + 0.002*"role" + 0.002*"scenes"
topic  2   words:  0.004*"characters" + 0.003*"life" + 0.003*"really" + 0.003*"films" + 0.003*"vampire" + 0.002*"many" + 0.002*"scenes" + 0.002*"see" + 0.002*"seems" + 0.002*"people"
topic  3   words:  0.003*"man" + 0.003*"life" + 0.003*"little" + 0.003*"characters" + 0.003*"love" + 0.002*"best" + 0.002*"many" + 0.002*"people" + 0.002*"&" + 0.002*"scenes"
topic  4   words:  0.003*"life" + 0.003*"family" + 0.003*"man" + 0.002*"best" + 0.002*"see" + 0.002*"characters" + 0.002*"films" + 0.002*"world" + 0.002*"go" + 0.002*"people"
topic  5   words:  0.006*"star" + 0.005*"trek" + 0.004*"funny" + 0.003*"action" + 0.003*"people" + 0.003*"work"

In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()

In [33]:
from nltk.tokenize import RegexpTokenizer
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)

In [34]:
nltk.download('stopwords')
stoplist = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
from collections import namedtuple
stemmer = PorterStemmer()

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [42]:
def PreprocessReviews(text,stop=[],stem=False):
    #print profile
    words = tknzr.tokenize(text)
    if stem:
        words_clean = [stemmer.stem(w) for w in [i.lower() for i in words if i not in stop]]
    else:
        words_clean = [i.lower() for i in words if i not in stop]
    return words_clean

Review = namedtuple('Review','words title tags')
dir_   = './data/txt_sentoken/'
do2vecstem = True
reviews_pos = []
cnt = 0

for filename in [f for f in os.listdir(dir_ +'pos/') if str(f)[0]!='.']:
    f   = open(dir_+'pos/'+filename,'r', encoding="ISO-8859-1")
    id_ = filename.split('.')[0].split('_')[1]
    reviews_pos.append(Review(PreprocessReviews(f.read(),stoplist,do2vecstem),moviedict[id_],['pos_'+str(cnt)]))
    cnt += 1

In [43]:
reviews_neg = []
cnt= 0
for filename in [f for f in os.listdir(dir_ +'neg/') if str(f)[0]!='.']:
    f = open(dir_ + 'neg/' + filename,'r')
    id_ = filename.split('.')[0].split('_')[1]
    reviews_neg.append(Review(PreprocessReviews(f.read(),stoplist,do2vecstem),moviedict[id_],['neg_'+str(cnt)]))
    cnt += 1

In [44]:
tot_reviews = reviews_pos + reviews_neg
#split in test training sets
def word_features(words):
    return dict([(word, True) for word in words])

In [45]:
negfeatures = [(word_features(r.words), 'neg') for r in reviews_neg]
posfeatures = [(word_features(r.words), 'pos') for r in reviews_pos]
portionpos = int(len(posfeatures)*0.8)
portionneg = int(len(negfeatures)*0.8)

In [46]:
print (portionpos,'-',portionneg)
trainfeatures = negfeatures[:portionneg] + posfeatures[:portionpos]
print (len(trainfeatures))
testfeatures = negfeatures[portionneg:]  + posfeatures[portionpos:]
#shuffle(testfeatures)

800 - 800
1600


In [47]:
from nltk.classify import NaiveBayesClassifier
#training naive bayes 
classifier = NaiveBayesClassifier.train(trainfeatures)
##testing
err = 0
print('test on: ',len(testfeatures)) 

test on:  400


In [ ]:
for r in testfeatures:
    sent = classifier.classify(r[0])
    if sent != r[1]:
        err +=1.
print ('error rate: ',err/float(len(testfeatures)))

In [ ]:
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from random import shuffle

#train bigram:
def bigrams_words_features(words, nbigrams=200,measure=BigramAssocMeasures.chi_sq):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams       = bigram_finder.nbest(measure, nbigrams)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])

negfeatures = [(bigrams_words_features(r.words,500), 'neg') for r in reviews_neg]
posfeatures = [(bigrams_words_features(r.words,500), 'pos') for r in reviews_pos]
portionpos  = int(len(posfeatures)*0.8)
portionneg  = int(len(negfeatures)*0.8)

In [ ]:
print(portionpos,'-',portionneg) 
trainfeatures = negfeatures[:portionpos] + posfeatures[:portionneg]
print(len(trainfeatures)) 
classifier = NaiveBayesClassifier.train(trainfeatures)

In [ ]:
##test bigram
testfeatures = negfeatures[portionneg:] + posfeatures[portionpos:]
shuffle(testfeatures)
err = 0
print('test on: ',len(testfeatures)) 

In [ ]:
for r in testfeatures:
    sent = classifier.classify(r[0])
    #print r[1],'-pred: ',sent
    if sent != r[1]:
        err +=1.
print('error rate: ',err/float(len(testfeatures))) 

In [ ]:
import nltk.classify.util, nltk.metrics
from nltk.probability import FreqDist, ConditionalFreqDist
tot_poswords = [val for l in [r.words for r in reviews_pos] for val in l]
tot_negwords = [val for l in [r.words for r in reviews_neg] for val in l]
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()

for word in tot_poswords:
    word_fd[word.lower()] +=1
    label_word_fd['pos'][word.lower()] +=1

for word in tot_negwords:
    word_fd[word.lower()] +=1
    label_word_fd['neg'][word.lower()] +=1

In [ ]:
pos_words = len(tot_poswords)
neg_words = len(tot_negwords)
tot_words = pos_words + neg_words
#select the best words in terms of information contained in the two classes pos and neg
word_scores = {}

for word, freq in word_fd.items():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
                (freq, pos_words), tot_words)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
                (freq, neg_words), tot_words)
    word_scores[word] = pos_score + neg_score

print('total: ',len(word_scores)) 
best = sorted(word_scores.items(), reverse=True)[:10000] # key=lambda (w,s): s
bestwords = set([w for w, s in best])

In [ ]:
#training naive bayes with chi square feature selection of best words
def best_words_features(words):
    return dict([(word, True) for word in words if word in bestwords])

In [ ]:
negfeatures = [(best_words_features(r.words), 'neg') for r in reviews_neg]
posfeatures = [(best_words_features(r.words), 'pos') for r in reviews_pos]
portionpos = int(len(posfeatures)*0.8)
portionneg = int(len(negfeatures)*0.8)
print(portionpos,'-',portionneg) 

In [ ]:
trainfeatures = negfeatures[:portionpos] + posfeatures[:portionneg]
print(len(trainfeatures)) 

In [ ]:
classifier = NaiveBayesClassifier.train(trainfeatures)
## test with feature chi square selection
testfeatures = negfeatures[portionneg:] + posfeatures[portionpos:]
shuffle(testfeatures)
err = 0
print('test on: ',len(testfeatures)) 

In [ ]:
for r in testfeatures:
    sent = classifier.classify(r[0])
    #print r[1],'-pred: ',sent
    if sent != r[1]:
        err +=1.
print('error rate: ',err/float(len(testfeatures))) 

In [ ]:
from gensim.models import Doc2Vec
import multiprocessing

shuffle(tot_reviews)
cores = multiprocessing.cpu_count()
vec_size = 500
model_d2v = Doc2Vec(dm=1, dm_concat=0, vec_size=vec_size, window=5, negative=0, hs=0, min_count=1, workers=cores)

In [ ]:
type(tot_reviews[0])

In [ ]:
# build vocab
model_d2v.build_vocab(tot_reviews)
# train
numepochs= 20
for epoch in range(numepochs):
    try:
        print('epoch %d' % (epoch))
        model_d2v.train(it, epochs=model.iter, total_examples=model.corpus_count)
#         model_d2v.train(tot_reviews)
#         model_d2v.alpha *= 0.99
#         model_d2v.min_alpha = model_d2v.alpha
    except (KeyboardInterrupt, SystemExit):
        break

In [ ]:
#split train,test sets
trainingsize = 2*int(len(reviews_pos)*0.8)
train_d2v    = np.zeros((trainingsize, vec_size))
train_labels = np.zeros(trainingsize)
test_size = len(tot_reviews)-trainingsize
test_d2v = np.zeros((test_size, vec_size))
test_labels = np.zeros(test_size)

In [ ]:
cnt_train = 0
cnt_test = 0
for r in reviews_pos:
    name_pos = r.tags[0]
    if int(name_pos.split('_')[1])>= int(trainingsize/2.):
        test_d2v[cnt_test] = model_d2v.docvecs[name_pos]
        test_labels[cnt_test] = 1
        cnt_test +=1
    else:
        train_d2v[cnt_train] = model_d2v.docvecs[name_pos]
        train_labels[cnt_train] = 1
        cnt_train +=1

In [ ]:
for r in reviews_neg:
    name_neg = r.tags[0]
    if int(name_neg.split('_')[1])>= int(trainingsize/2.):
        test_d2v[cnt_test] = model_d2v.docvecs[name_neg]
        test_labels[cnt_test] = 0
        cnt_test +=1
    else:
        train_d2v[cnt_train] = model_d2v.docvecs[name_neg]       
        train_labels[cnt_train] = 0
        cnt_train +=1

In [ ]:
#train log regre
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(train_d2v, train_labels)
'accuracy:',classifier.score(test_d2v,test_labels)

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_d2v, train_labels)
'accuracy:',clf.score(test_d2v,test_labels)

In [ ]:
#svm linear
clf = SVC(kernel='linear')
clf.fit(train_d2v, train_labels)
clf.score(test_d2v,test_labels)